In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping

# Load the Diagnostics.xlsx data
diagnostics_file = "../../../Datasets/12-lead electrocardiogram database/Diagnostics.xlsx"
diagnostics_df = pd.read_excel(diagnostics_file)

# Rename "SA" to "SI" in the "Rhythm" column
diagnostics_df["Rhythm"] = diagnostics_df["Rhythm"].replace("SA", "SI")

# Drop rows with any missing values
diagnostics_df = diagnostics_df.dropna()

# Encode "Gender" column: 0 for "MALE" and 1 for "FEMALE"
diagnostics_df["Gender"] = diagnostics_df["Gender"].map({"MALE": 0, "FEMALE": 1})

# Merge specified labels
merge_mapping = {
    "AF": "AFIB", "AFIB": "AFIB",
    "SVT": "GSVT", "AT": "GSVT", "SAAWR": "GSVT", "ST": "GSVT", "AVNRT": "GSVT", "AVRT": "GSVT",
    "SB": "SB",
    "SR": "SR", "SI": "SR"
}
diagnostics_df["Rhythm"] = diagnostics_df["Rhythm"].map(merge_mapping)

# Separate features and labels
features = diagnostics_df.drop(columns=["FileName", "Rhythm", "Beat"]).values
labels = diagnostics_df["Rhythm"].values  # Using "Rhythm" as the target variable

# Convert features to float32
features = features.astype("float32")

# Encode labels as one-hot with merged classes
unique_labels = np.unique(labels)
label_map = {label: index for index, label in enumerate(unique_labels)}
labels_encoded = to_categorical([label_map[label] for label in labels])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)

2024-11-08 22:48:05.375120: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-08 22:48:05.386022: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-08 22:48:05.389488: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-08 22:48:05.398274: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-08 22:48:05.939381: W tensorflow/compiler/tf2

In [2]:
# Reshape the data for CNN input: we will use 2D convolution, so we need to reshape (samples, features) into (samples, time_steps, channels)
# Here, each ECG feature (e.g., Ventricular Rate, Atrial Rate) will be treated as a feature in the 2D matrix, and time is along one axis.
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1, 1)  # (samples, features, 1, 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1, 1)  # (samples, features, 1, 1)

# Define the CNN+MLP hybrid model with 2D convolution
model = Sequential([
    # 2D Convolutional layer for feature extraction
    Conv2D(64, (3, 1), activation='relu', input_shape=(X_train_reshaped.shape[1], 1, 1)),
    MaxPooling2D((2, 1)),
    Dropout(0.3),

    # Flatten the output from CNN layer and pass it to dense layers
    Flatten(),

    # MLP layers for classification
    Dense(128, activation="relu"),
    Dropout(0.3),
    Dense(64, activation="relu"),
    Dense(labels_encoded.shape[1], activation="softmax")  # Output layer with the number of merged classes
])

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Set up early stopping
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train_reshaped, y_train, validation_data=(X_test_reshaped, y_test), epochs=20, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
y_pred = model.predict(X_test_reshaped)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Map back to original labels for a readable report
label_names = [label for label, index in sorted(label_map.items(), key=lambda item: item[1])]
print("\nClassification Report:\n")
print(classification_report(y_test_classes, y_pred_classes, target_names=label_names))

Epoch 1/20


/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1731084487.775334  342805 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1731084487.808823  342805 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:

175/267 ━━━━━━━━━━━━━━━━━━━━ 0s 866us/step - accuracy: 0.3718 - loss: 11.3976

I0000 00:00:1731084490.107543  342952 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


267/267 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.3951 - loss: 8.9804 - val_accuracy: 0.5554 - val_loss: 0.9086
Epoch 2/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5313 - loss: 1.0807 - val_accuracy: 0.7033 - val_loss: 0.8081
Epoch 3/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 901us/step - accuracy: 0.5968 - loss: 0.9063 - val_accuracy: 0.7695 - val_loss: 0.7774
Epoch 4/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 832us/step - accuracy: 0.6406 - loss: 0.8227 - val_accuracy: 0.7615 - val_loss: 0.7105
Epoch 5/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 723us/step - accuracy: 0.6730 - loss: 0.7809 - val_accuracy: 0.7704 - val_loss: 0.6676
Epoch 6/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 859us/step - accuracy: 0.7033 - loss: 0.7117 - val_accuracy: 0.7507 - val_loss: 0.6199
Epoch 7/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7113 - loss: 0.6898 - val_accuracy: 0.8305 - val_loss: 0.5521
Epoch 8/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step - accuracy: 0.7071 - loss: 0.6974 - val_accuracy: 0.